In [1]:
import org.zouzias.spark.lucenerdd._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.12:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1653145042528)
SparkSession available as 'spark'


import org.zouzias.spark.lucenerdd._


In [196]:
val activities = spark.read.parquet("/tmp/phone-calls/_activities")

activities: org.apache.spark.sql.DataFrame = [id: int, datetime: string ... 10 more fields]


In [197]:
val calls = spark.read.parquet("/tmp/phone-calls/_calls")

calls: org.apache.spark.sql.DataFrame = [id: string, datetime: string ... 10 more fields]


In [216]:
import org.apache.spark.sql.Row

def searchQuery: Row => String = {
    case row => {
        val search = Seq(
            // "datetime",
            "year",
            "month",
            "day",
            "hour",
            // "minute",
            "user",
            // "phone"
            "prefix",
            "main",
            // "suffix"
        )
        .map(key => key -> row.get(row.fieldIndex(key)))
        .filter(_._2 != null)
        .map { case (key, value) =>
            val string = value.toString
            (key, string)
        }
        .map { case (key, value) => s"(${key}:(${value}))" }
        .mkString(" AND ")

    if (search.nonEmpty) search else "*:*"
    }
}

searchQuery(activities.sample(false, 1e-2, 0).first)

import org.apache.spark.sql.Row
searchQuery: org.apache.spark.sql.Row => String
res132: String = (year:(2021)) AND (month:(8)) AND (day:(20)) AND (hour:(8)) AND (user:(cierra kilpatrick)) AND (prefix:(415)) AND (main:(359))


In [167]:
searchQuery(calls.sample(false, 1e-2, 42).first)

res104: String = (datetime:(2022-05-11 14-03-35)) OR (user:(toni engle)) OR (phone:(800 826 0270))


In [217]:
val luceneRDD = LuceneRDD(activities.where("direction = 'outbound'"))

val results = luceneRDD.linkDataFrame(calls.where("direction = 'outbound'"), searchQuery, 1).filter(_._2.nonEmpty)

luceneRDD: org.zouzias.spark.lucenerdd.LuceneRDD[org.apache.spark.sql.Row] = LuceneRDD[1454] at RDD at LuceneRDD.scala:54
results: org.apache.spark.rdd.RDD[(org.apache.spark.sql.Row, Array[org.apache.spark.sql.Row])] = MapPartitionsRDD[1471] at filter at <console>:118


In [184]:
// val luceneRDD = LuceneRDD(calls.where("direction = 'outbound'"))

// val results = luceneRDD.linkDataFrame(activities.where("direction = 'outbound'"), searchQuery, 1).filter(_._2.nonEmpty)

luceneRDD: org.zouzias.spark.lucenerdd.LuceneRDD[org.apache.spark.sql.Row] = LuceneRDD[1188] at RDD at LuceneRDD.scala:54
results: org.apache.spark.rdd.RDD[(org.apache.spark.sql.Row, Array[org.apache.spark.sql.Row])] = MapPartitionsRDD[1205] at filter at <console>:106


In [171]:
// calls.where("direction = 'outbound'").count
calls.where("direction = 'outbound'").where("phone is null").count

res107: Long = 33


In [191]:
// results.flatMap { case (key, values) =>
//     values.map { value =>
//         (key, value)
//    }
// }
// .take(5)
// .foreach(println)

In [218]:
results.count

res133: Long = 1351


In [194]:
// activities.where("direction = 'outbound'").count

In [219]:
%%time
results.flatMap { case (key, values) =>
    values.map { value =>
        (key.getAs[String]("id"), value.getAs[Int]("id"), value.getAs[Float]("__score__").toDouble)
   }
}
.toDF("id", "_id", "_score")
// .where("id != _id")
.write
.mode("overwrite")
.parquet(s"/tmp/phone-calls/outbound")


val bridge = spark.read.parquet("/tmp/phone-calls/outbound")
// bridge.createOrReplaceTempView("bridge")

Time: 0.911113977432251 seconds.



bridge: org.apache.spark.sql.DataFrame = [id: string, _id: int ... 1 more field]


In [190]:
// %%time
// results.flatMap { case (key, values) =>
//     values.map { value =>
//         (key.getAs[Int]("id"), value.getAs[String]("id"), value.getAs[Float]("__score__").toDouble)
//    }
// }
// .toDF("_id", "id", "_score")
// .select("id", "_id", "_score")
// // .where("id != _id")
// .write
// .mode("overwrite")
// .parquet(s"/tmp/phone-calls/outbound")


// val bridge = spark.read.parquet("/tmp/phone-calls/outbound")
// // bridge.createOrReplaceTempView("bridge")

In [165]:
// bridge.show(false)
bridge.orderBy("_score").show(false)

+------------------------------------+--------+------------------+
|id                                  |_id     |_score            |
+------------------------------------+--------+------------------+
|3ec429f8-b336-439a-a17a-14da1f558daa|47588729|6.120454788208008 |
|c8052032-8f34-449f-b4a2-721886b458f4|47952017|6.811224937438965 |
|ca4f5bae-2a4a-4f57-bcba-4378954e6ace|35664829|6.928653240203857 |
|bc00e2f4-8afc-451c-9450-2b4bb6855d27|46401194|7.249926567077637 |
|49e86779-6a5a-4e67-889d-3d2561231d4e|47627392|7.474926948547363 |
|f18a27c2-9345-44d5-8d9d-25fc1be40f71|46724665|7.549892425537109 |
|aa181471-fddb-4e56-9b13-8c3b4ba6a9de|47838685|7.575432300567627 |
|953a2986-c92d-498f-8c49-bdb0b2612719|47835021|7.687255859375    |
|8068edd4-98f4-49f3-9314-23e7c9f7a36e|47776099|8.715286254882812 |
|e9f84453-cb09-46ef-a30f-ee0d86940648|33671097|9.518377304077148 |
|786552c8-9c95-4e75-b2a6-f24a13bb6072|46932568|9.966015815734863 |
|812e6d1e-9e8f-4b31-9ebd-1d63178b9f73|40445314|10.228724479675

### Testing

In [118]:
val table = spark.read.parquet("/tmp/phone-calls/VendorContacts").selectExpr("ObjectID as ID", "Phone")

table: org.apache.spark.sql.DataFrame = [ID: int, Phone: string]


In [112]:
import org.apache.spark.sql.Row

// +18287377552
def searchQuery: Row => String = {
    case row => {
        val search = Seq("Phone")
            .map(key => key -> row.get(row.fieldIndex(key)))
            .filter(_._2 != null)
            .map { case (key, value) =>
                val string = value
                    .toString
                    .replaceAll("[^0-9]", "")
                    .split("")
                    .mkString("-")
                (key, string)
            }
            .filter(_._2.nonEmpty)
            .map { case (key, value) => s"(${key}:(${value}))" }
            .mkString("-")

        if (search.nonEmpty) search else "*:*"
    }
}

// searchQuery(table.sample(0.01).first)

import org.apache.spark.sql.Row
searchQuery: org.apache.spark.sql.Row => String


In [121]:
val sample = Seq("8287377552").toDF("Phone")
// val sample = Seq("1 828 737 5752").toDF("Phone")

sample: org.apache.spark.sql.DataFrame = [Phone: string]


In [108]:
val table = Seq("1 828 737 7552").toDF("Phone")

table: org.apache.spark.sql.DataFrame = [Phone: string]


In [113]:
searchQuery(sample.first)

res66: String = (Phone:(1-8-2-8-7-3-7-5-7-5-2))


In [114]:
searchQuery(table.first)

res67: String = (Phone:(1-8-2-8-7-3-7-7-5-5-2))


In [122]:
val luceneRDD = LuceneRDD(table)

val results = luceneRDD.linkDataFrame(sample, searchQuery, 8)

luceneRDD: org.zouzias.spark.lucenerdd.LuceneRDD[org.apache.spark.sql.Row] = LuceneRDD[412] at RDD at LuceneRDD.scala:54
results: org.apache.spark.rdd.RDD[(org.apache.spark.sql.Row, Array[org.apache.spark.sql.Row])] = MapPartitionsRDD[427] at map at LuceneRDD.scala:248


In [123]:
// results.filter(_._2.nonEmpty).take(8)

results.take(1)

// _table.show()

res70: Array[(org.apache.spark.sql.Row, Array[org.apache.spark.sql.Row])] = Array(([8287377552],Array([8007968707, opt1, 2, 7,31410184,0,11.4012165,128156], [8,29617332,0,11.135309,109245], [8007435637 opt 7,31339053,0,9.047127,221824], [8008888018 opt 7,34904976,0,9.047127,140100], [(310) 547-4973 ext 7,34513688,0,6.329067,241409], [8003321375 opt 5,29603247,0,5.921568,269237], [18007332383 Opt. 5,29542997,0,5.921568,36788], [4126427600 option 2,29524908,0,4.802253,21017])))
